In [1]:
#import packages
import matplotlib.image as mpimg
import numpy as np
import cv2
import time
import os
from lesson_functions import *
from skimage.feature import hog
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.externals import joblib


In [2]:
def loadData(folder):
    cars = []
    notcars = []
    for dirpath, dirnames, files in os.walk(folder):
        if 'non-vehicles' in dirpath:
            for image in files:
                notcars.append(os.path.join(dirpath, image))
        else:
            for image in files:
                cars.append(os.path.join(dirpath, image))
    return cars, notcars

# Define a function to return some characteristics of the dataset 
def dataLook(car_list, notcar_list):
    data_dict = {}
    # Define a key in data_dict "nCars" and store the number of car images
    data_dict["nCars"] = len(car_list)
    # Define a key "nNotCars" and store the number of notcar images
    data_dict["nNotCars"] = len(notcar_list)
    # Read in a test image, either car or notcar
    # Define a key "imageShape" and store the test image shape 3-tuple
    sample = cv2.imread(car_list[0])
    data_dict["imageShape"] = sample.shape
    # Define a key "dataType" and store the data type of the test image.
    data_dict["dataType"] = sample.dtype
    # Return data_dict
    return data_dict

In [3]:
carsList, notCarsList = loadData("data")
dataInfo = dataLook(carsList, notCarsList)

print(dataInfo["nCars"], "cars,", dataInfo["nNotCars"], "non-cars")
print("Image size =", dataInfo["imageShape"], ", Image type =", dataInfo["dataType"])


8792 cars, 8968 non-cars
Image size = (64, 64, 3) , Image type = uint8


In [4]:
# Tweak these parameters and see how the results change.
color_space = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"
spatial_size = (32, 32) # Spatial binning dimensions
hist_bins = 32    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off

carFeatures = extract_features(carsList, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
notCarFeatures = extract_features(notCarsList, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)

C:\Users\Ong Whee Cheng\Miniconda3\envs\carnd-term1\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


In [5]:
X = np.vstack((carFeatures, notCarFeatures)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(carFeatures)), np.zeros(len(notCarFeatures))))

# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=rand_state)

print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))
# Use a linear SVC 
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()

# Save trained model and scaler
print("Model saved to", joblib.dump(svc, "svm\svc.pkl"))
print("Scaler saved to", joblib.dump(X_scaler, "svm\scaler.pkl"))

Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 8460
5148.53 Seconds to train SVC...
Test Accuracy of SVC =  0.989
Model saved to ['svm\\svc.pkl']
Scaler saved to ['svm\\scaler.pkl']
